In [1]:
import pandas as pd
import re
import joblib
import os

In [2]:
from ipynb.fs.full. datasets import *

In [3]:
combined.columns = combined.columns.str.replace('[^a-zA-Z0-9]', '', regex = True)
#print(combined)
for col in combined.columns:
    if combined[col].dtype == 'object':
        combined[col] = combined[col].str.lower()

In [4]:
#combined.head(5)

In [5]:
def preprocess_data(combined):
    combined['title'] = combined['title'].apply(clean_text)
    optional_text_columns = ['Description', 'tag']
    for col in optional_text_columns:
        if col in combined.columns:
            combined[col] = combined[col].apply(clean_text)
    return combined
    

In [6]:
course['type'] = 'course'
book['type'] = 'book'

In [7]:
combined['combined_data'] = (combined['title'] + ' ' + combined['Description'] + ' ' + combined['tag'].fillna(''))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    stop_words='english', 
    max_features=5000,
    ngram_range=(1,2)
)

In [9]:
tfidf_matrix = tfidf.fit_transform(combined['combined_data'])
print(tfidf_matrix.shape)

(13533, 5000)


In [10]:
#combined.tail(5)

In [11]:
#joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
query = "python programming for beginners"
query_vector = tfidf.transform([query])
similarity_scores = cosine_similarity(query_vector, tfidf_matrix)

In [13]:
os.makedirs("models", exist_ok=True)
joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')
joblib.dump(tfidf_matrix, "models/tfidf_matrix.pkl")  # Add this line!


['models/tfidf_matrix.pkl']

In [14]:
#for testing purpose
#feature_names = tfidf.get_feature_names_out()
#print("First features:", feature_names[200:500])

In [15]:
#os.makedirs("data", exist_ok=True)
#combined.to_csv("data/processed_data.csv")

In [16]:
os.makedirs("data", exist_ok=True)
combined.to_csv("data/processed_data.csv")

In [17]:
df = pd.read_csv("data/processed_data.csv")
print(df)
df.head(5)

       Unnamed: 0 Unnamed0                                              title  \
0               0      NaN                              to kill a mockingbird   
1               1      NaN  harry potter and the philosopher’s stone (harr...   
2               2      NaN                                pride and prejudice   
3               3      NaN                          the diary of a young girl   
4               4      NaN                                        animal farm   
...           ...      ...                                                ...   
13528       13673           learn jquery from scratch - master of javascri...   
13529       13674           how to design a wordpress website with no codi...   
13530       13675                               learn and build using polymer   
13531       13676           css animations: create amazing effects on your...   
13532       13677           using modx cms to build websites: a beginner's...   

                           

,Unnamed: 0,Unnamed0,title,Description,tag,url,type,price,level,contentduration,combined_data
0,0,NaN,to kill a mockingbird,the unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical fiction', ...",https://www.goodreads.com/book/show/2657.to_ki...,book,NaN,NaN,NaN,to kill a mockingbird the unforgettable novel ...
1,1,NaN,harry potter and the philosopher’s stone (harr...,harry potter thinks he is an ordinary boy - un...,"['fantasy', 'fiction', 'young adult', 'magic',...",https://www.goodreads.com/book/show/72193.harr...,book,NaN,NaN,NaN,harry potter and the philosopher’s stone (harr...
2,2,NaN,pride and prejudice,"since its immediate success in 1813, pride and...","['classics', 'fiction', 'romance', 'historical...",https://www.goodreads.com/book/show/1885.pride...,book,NaN,NaN,NaN,pride and prejudice since its immediate succes...
3,3,NaN,the diary of a young girl,discovered in the attic in which she spent the...,"['classics', 'nonfiction', 'history', 'biograp...",https://www.goodreads.com/book/show/48855.the_...,book,NaN,NaN,NaN,the diary of a young girl discovered in the at...
4,4,NaN,animal farm,librarian's note: there is an alternate cover ...,"['classics', 'fiction', 'dystopia', 'fantasy',...",https://www.goodreads.com/book/show/170448.ani...,book,NaN,NaN,NaN,animal farm librarian's note: there is an alte...
